In [2]:
import numpy as np
import numpy.random as random
import pandas as pd
import scipy as sp
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

%precision 3

import sklearn
import requests, zipfile
import io

import statsmodels.api as sm

__정규화 선형회귀__

---


정규화(regularized) 선형회귀 방법은 선형회귀 계수(weight)에 대한 제약 조건을 추가함으로써 모형이 과도하게 최적화되는 현상, 즉 과최적화를 막는 방법이다. Regularized Method, Penalized Method, Contrained Least Squares 이라고도 불리운다.

모형이 과도하게 최적화되면 모형 계수의 크기도 과도하게 증가하는 경향이 나타난다. 따라서 정규화 방법에서 추가하는 제약 조건은 일반적으로 계수의 크기를 제한하는 방법이다. 일반적으로 다음과 같은 세가지 방법이 사용된다.

* Ridge 회귀모형
* Lasso 회귀모형
* Elastic Net 회귀모형

<p>&nbsp;</p>

__Ridge 회귀모형__

---

Ridge 회귀모형에서는 가중치들의 제곱합(squared sum of weights)을 최소화하는 것을 추가적인 제약 조건으로 한다.

$$
w = \text{arg}\min_w \left( \sum^N_{i=1}e^2_i + \lambda \sum^M_{j=1}w^2_j\right)
$$

$\lambda$는 기존의 잔차 제곱합과 추가적 제약 조건의 비중을 조절하기 위한 하이퍼 모수(hyper parameter)이다. $\lambda$가 크면 정규화 정도가 커지고 가중치의 값들이 작아진다. $\lambda$가 작아지면 정규화 정도가 작아지며 $\lambda$가 0이 되면 일반적인 선형 회귀모형이 된다.

<p>&nbsp;<p>

__Lasso 회귀모형__

---

Lasso(Least Absolute Shrinkage and Selection Operator) 회귀모형은 가중치의 절대값의 합을 최소화하는 것을 추가적인 제약 조건으로 한다.

$$
w = \text{arg} \min_w \left( \sum^N_{i=1} e^2_i + \lambda \sum^M_{j=1} |w_j| \right)
$$

<p>&nbsp;</p>

__Elastic Net 회귀모형__

---

Elastci Net 회귀모형은 가중치의 절대값의 합과 제곱합을 동시에 제약 조건으로 가지는 모형이다.

$$
w = \text{arg} \min_w \left( \sum^N_{i=1} e^2_i + \lambda_1 \sum^M_{j=1} |w_j| + \lambda_2 \sum^M_{j=1} w^2_j \right)
$$

$\lambda_1$, $\lambda_2$ 두 개의 하이퍼 모수를 가진다.

<p>&nbsp;</p>

__statsmodels의 정규화 회귀모형__

---

statsmodels 패키지는 OLS 선형 회귀모형 클래스의 `fit_regularized` 메서드를 사용하여 Elastic Net 모형 계수를 구할 수 있다.

하이퍼 모수는 다음과 같이 모수 $\text{alpha}$와 $\text{L1} \_ \text{wt}$로 정의된다.

$$
0.5 \times \text{RSS} / \text{alpha} \times (0.5 \times (1 - \text{L1} \_ \text{wt}) \sum w^2_i + \text{L1} \_ \text{wt} \sum |w_i|)

In [4]:
def make_nonlinear(seed=0):
    np.random.seed(seed)
    n_samples = 30
    X = np.sort(np.random.rand(n_samples))
    y = np.sin(2 * np.pi * X) + np.random.randn(n_samples) * 0.1
    X = X[:, np.newaxis]
    return (X, y)

X, y = make_nonlinear()
dfX = pd.DataFrame(X, columns=["x"])
dfX = sm.add_constant(dfX)
dfy = pd.DataFrame(y, columns=["y"])
df = pd.concat([dfX, dfy], axis = 1)

In [5]:
model = sm.OLS.from_formula(
     "y ~ x + I(x**2) + I(x**3) + I(x**4) + I(x**5) + I(x**6) + I(x**7) + I(x**8) + I(x**9)", data=df)

result1 = model.fit()
result1.params

Intercept       -0.169863
x               25.735773
I(x ** 2)     -428.141683
I(x ** 3)     3866.723113
I(x ** 4)   -18340.939658
I(x ** 5)    49326.072530
I(x ** 6)   -78884.743049
I(x ** 7)    74538.645130
I(x ** 8)   -38453.132179
I(x ** 9)     8350.254983
dtype: float64